In [ ]:
import random
from copy import deepcopy
from typing import Literal

import pandas as pd
import numpy as np
from scipy.special import logsumexp

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
eps = 1e-9

SEED = 18092025
random.seed(SEED)
np.random.seed(SEED)

[Датасет](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success)

In [ ]:
try:
    df = pd.read_csv('../datasets/data.csv', delimiter=';')
except Exception:
    print('No such file')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

pd.set_option('display.expand_frame_repr', False)


# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')
# pd.reset_option('display.width')
# pd.reset_option('display.expand_frame_repr')

---

## EDA

In [ ]:
def display_nans(df):
    nans_per_col = [(col, df[col].isna().sum(), df[col].isna().sum() / df.shape[0] * 100) for col in df.columns]
    dtype = [('col_name', 'U20'), ('nans', int), ('nans_perc', float)]
    nans_per_col = np.array(nans_per_col, dtype=dtype)
    nans_per_col = nans_per_col[nans_per_col['nans'] > 0]
    nans_per_col = np.sort(nans_per_col, order='nans')

    if nans_per_col.shape[0] == 0:
        print('No nans in the dataset')
        return

    df_show = pd.DataFrame(nans_per_col[::-1])
    display(df_show.style.background_gradient(cmap='Blues'))
    
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))

    y_pos = np.arange(len(nans_per_col))
    
    ax.barh(y_pos, nans_per_col['nans_perc'], alpha=0.8, edgecolor='black', linewidth=1) 
    ax.set_yticks(y_pos, labels=nans_per_col['col_name'])
    ax.set_xlabel('Nans, %', fontsize=14)
    ax.set_title('Nans rate for each column', fontsize=16)
    ax.set_xlim(0, min(np.max(df_show['nans_perc']) + 5.0, 100.0))
    ax.tick_params(axis='both', which='major', labelsize=11)
    ax.grid(axis='x', linestyle='--', linewidth=0.5)
    
    plt.show()

In [ ]:
display(df.head())
print('Dataset shape: ', df.shape)

In [ ]:
def col_names_transform(col_name: str) -> str:
    res_name = col_name.strip().replace("\t", "").replace(' ', '_').lower()
    return res_name

In [ ]:
df.columns = map(col_names_transform, df.columns.values)
df.columns

In [ ]:
df.describe()

In [ ]:
# TODO: добавить другие стат. показатели 

In [ ]:
display_nans(df)

In [ ]:
df.dtypes

In [ ]:
df['target'].value_counts(normalize=True).to_frame().T

---

## Подготовка данных

In [ ]:
X, y = df.drop(columns=['target']), df['target']
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=SEED, shuffle=True, stratify=y)
X_train, X_val, y_train, y_val   = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED, shuffle=True, stratify=y_train)
y_train.shape, y_val.shape, y_test.shape

In [ ]:
std_scaler = StandardScaler()
X_train_scaled = std_scaler.fit_transform(X_train)
X_val_scaled   = std_scaler.transform(X_val)
X_test_scaled  = std_scaler.transform(X_test)
X_train_scaled[0, :5]

In [ ]:
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train.values)
y_val_enc   = label_encoder.transform(y_val.values)
y_test_enc  = label_encoder.transform(y_test.values)
y_train[:5].values, y_train_enc[:5], label_encoder.classes_

---

## Модель линейной классификации

In [ ]:
class RunningAverage:
    def __init__(self):
        self.value = 0.0
        self.count = 0

    def update(self, new_value):
        self.count += 1
        self.value = (1 / self.count) * new_value + (1 - 1 / self.count) * self.value
        return self.value
    

class EMA:
    def __init__(self, l=0.1):
        self.value = None
        self.l = l

    def update(self, new_value):
        if self.value is not None:
            self.value = self.l * new_value + (1 - self.l) * self.value
        else:
            self.value = new_value
        return self.value

In [ ]:
class LogRegNumpy():

    def __init__(
        self,
        initial_weights: list[list[float]] = None, # (n_features, n_classes)
        initial_bias:    list[float] = None,       # (1, n_classes)
        tolerance:       float = 1e-4,
        early_stop: bool = False,
        n_startup_rounds: int = 50,
        early_stop_rounds: int = 50,
        random_seed: int = SEED
    ):
        self.weights = (np.array(initial_weights) if initial_weights is not None 
                        else np.array([]))
        self.bias = (np.array(initial_bias) if initial_bias is not None 
                     else np.array([]))
        self.tolerance = tolerance
        self.early_stop = early_stop
        self.n_startup_rounds = n_startup_rounds
        self.early_stop_rounds = early_stop_rounds
        self.random_seed = random_seed
        self.eps = 1e-9

        self.rng_ = np.random.default_rng(seed=random_seed)

        # для рекуррентной оценки
        self.rec_value = None
        self.rec_count = 0
        self.rec_history = []

    
    def fit(
        self,
        features:       list[list[float]],
        labels:         list[int],
        learning_rate:  float = 1e-3,
        total_steps:    int = 1000,
        batch_size:     int | None = None, # None = full dataset
        gd_algo:        Literal['gd', 'sgd'] = 'sgd',
        momentum:       float = 0.0,
        l2:             float = 0.0,
        optim_step:     bool = False,
        init_strategy:  Literal['normal', 'corr', 'multistart'] = 'normal',
        return_weights_history:  bool = False,
        verbose:                 bool = False,
        # рекуррентная оценка функции потерь
        rec_mode:                Literal['off','mean','ema'] = 'off',
        ema_lambda:              float = 0.1,
        # стратегия сэмплирования
        sampling_mode:           Literal['uniform','by_margin'] = 'uniform',
        shuffle:                 bool = True,
        sampling_tau:            float = 0.2,
        sampling_min_prob:       float = 0.01,
        refresh_rate:            int = 100,
        steps_per_epoch:         int | None = 100, # для логики логирования
    ) -> None | list[list[float]]:

        X = (np.array(features, dtype=np.float32).squeeze()
            if not isinstance(features, np.ndarray)
            else deepcopy(features).astype(np.float32, copy=False))
        y = (np.array(labels, dtype=np.int64).squeeze()
            if not isinstance(labels,  np.ndarray)
            else deepcopy(labels).astype(np.int64,  copy=False))
        if X.ndim == 1:
            X = X[np.newaxis, :]
        N = X.shape[0]

        if gd_algo == 'gd':
            batch_size = None

        if batch_size is None:
            batch_size = N

        if steps_per_epoch is None:
            steps_per_epoch = (N + batch_size - 1) // batch_size

        # --- init
        self._init_weights(X, y, init_strategy)
        if return_weights_history:
            weights_values = [self.weights.copy()]
        Vdw = np.zeros_like(self.weights)
        Vdb = np.zeros_like(self.bias)
        self._rec_reset()
        self.loss_values = []
        no_improvement_counter = 0
        rng = self.rng_
            
        def uniform_next_batch_stateful():
            perm = rng.permutation(N) if shuffle else np.arange(N, dtype=np.int64)
            ptr = 0

            def get_batch():
                nonlocal perm, ptr
                if ptr >= N:
                    perm = rng.permutation(N) if shuffle else np.arange(N, dtype=np.int64)
                    ptr = 0

                remaining = N - ptr
                take = batch_size if batch_size <= remaining else remaining

                idx = perm[ptr:ptr + take]
                ptr += take
                return idx

            return get_batch

        margin_probs = lambda: self._margin_sampling_probs(
            X, y, use_abs=True, tau=sampling_tau, min_prob=sampling_min_prob
        )

        # ---- training loop (one unified path)
        step = 0
        block_loss_sum = 0.0
        block_count = 0

        if sampling_mode == 'uniform':
            next_uniform_batch = uniform_next_batch_stateful()
        elif sampling_mode == 'by_margin':
            probs = margin_probs()
            next_uniform_batch = None
        else:
            raise ValueError("sampling_mode must be 'uniform' or 'by_margin'")

        while step < total_steps:
            if sampling_mode == 'uniform':
                batch_idx = next_uniform_batch()
            else:  # by_margin
                if step % refresh_rate == 0:
                    probs = margin_probs()
                batch_idx = rng.choice(N, size=batch_size, replace=True, p=probs)

            xi = X[batch_idx, :]
            yi = y[batch_idx]

            # forward / loss
            logits = self.forward(xi)
            loss   = self._loss_fn_opt(yi, logits, reduction=None)

            # Self-Normalized Importance Sampling Loss
            if sampling_mode == 'by_margin':
                pi = probs[batch_idx]
                sample_weights = 1.0 / np.clip(pi, 1e-12, None)
                loss = (loss * sample_weights).sum() / sample_weights.sum()
            else:
                sample_weights = None
                loss = loss.mean()
                
            block_loss_sum += loss
            block_count    += 1

            # recurrent quality update
            rec_val = self._rec_update(loss, mode=rec_mode, ema_lambda=ema_lambda)
            self.rec_history.append(rec_val)

            # gradients
            w_grad, b_grad = self._gradient(xi, yi, logits)
            # L2 - regularization
            if l2 > 0.0:
                w_grad += l2 * self.weights

            # momentum (EMA style)
            Vdw = momentum * Vdw - (1.0 - momentum) * w_grad
            Vdb = momentum * Vdb - (1.0 - momentum) * b_grad
            
            if optim_step:
                learning_rate = self._line_search_backtracking(
                    xi, yi, w_grad, b_grad, Vdw, Vdb, l2=l2
                )

            # update
            self.weights += learning_rate * Vdw
            self.bias    += learning_rate * Vdb

            step += 1

            # logging once per “epoch-sized” number of steps
            if block_count >= steps_per_epoch:
                mean_block_loss = block_loss_sum / block_count
                self.loss_values.append(mean_block_loss)

                # # recurrent quality update
                # rec_val = self._rec_update(mean_block_loss, mode=rec_mode, ema_lambda=ema_lambda)
                # self.rec_history.append(rec_val)

                if return_weights_history:
                    weights_values.append(self.weights.copy())
                if verbose:
                    print(f"step {step:6d} | block_loss={mean_block_loss:.6f} | batch_size={batch_size} | mode={sampling_mode}")

                block_loss_sum = 0.0
                block_count = 0

            # early stopping on monitored series (smoothed if rec_mode != 'off')
            if self.early_stop and step > self.n_startup_rounds + 1:
                if 0 < (self.rec_history[-2] - self.rec_history[-1]) < self.tolerance:
                    no_improvement_counter += 1
                    if no_improvement_counter >= self.early_stop_rounds:
                        if verbose:
                            print(f"Early stopping at step {step}")
                        break
                else:
                    no_improvement_counter = 0

        if return_weights_history:
            return np.array(weights_values)
        
        
    def _line_search_backtracking(
        self,
        X, y, 
        grad_w: np.ndarray, grad_b: np.ndarray, 
        dir_w:  np.ndarray, dir_b:  np.ndarray,
        step: float = 1.0, 
        alpha: float = 1e-4, 
        beta: float = 0.5, 
        tol: float = 1e-8,
        l2: float = 0.0,
        default_lr: float = 1e-9,
        eps: float = 1e-12
    ):
        logits_0 = self.forward(X)
        loss_0 = self._loss_fn_opt(y, logits_0, reduction='mean')
        if l2 > 0.0:
            loss_0 += l2 * np.sum(np.pow(self.weights, 2)) * 0.5
        
        W0 = self.weights
        b0 = self.bias

        # directional derivatives
        dw = (grad_w * dir_w).sum()
        db = (grad_b * dir_b).sum()
        dd = dw + db
        
        if l2 > 0.0:
            dd += l2 * (W0 * dir_w).sum()

        if dd >= eps:
            if np.allclose(dir_w, -grad_w) and np.allclose(dir_b, -grad_b):
                return 0.0
            # switching to regular gradient descent
            np.copyto(dst=dir_w, src=-grad_w); np.copyto(dst=dir_b, src=-grad_b)
            dd = (grad_w * dir_w).sum() + (grad_b * dir_b).sum()
            if l2 > 0.0:
                dd += l2 * (W0 * dir_w).sum()
            if dd >= eps:
                return 0.0

        t = step

        # TODO: добавить оптимизации через раскрытие logits_t и l2_norm и предварительного подсчета неизменных членов
        while t > tol:
            Wt = W0 + t * dir_w
            bt = b0 + t * dir_b
            logits_t = np.matmul(X, Wt) + bt
            loss_t = self._loss_fn_opt(y, logits_t, reduction='mean')
            if l2 > 0.0:
                loss_t += l2 * np.sum(np.pow(Wt, 2)) * 0.5

            if loss_t <= loss_0 + alpha * t * dd:
                return t
            
            t *= beta

        return default_lr
        

    def predict(self, features: list[list[float]]):
        X = (np.array(features).squeeze() if not isinstance(features, np.ndarray) 
             else deepcopy(features).astype(np.float32, copy=False))
        if X.ndim == 1:
            X = X[np.newaxis, :]
        logits = self.forward(X) # (n_samples, n_classes)
        probs  = self._softmax(logits) # не обязательно
        return np.argmax(probs, axis=1)
    
    def _create_onehot_target(self, y: np.array):
        ohe_enc = OneHotEncoder(categories=[np.unique(y)], sparse_output=False)
        y_enc = ohe_enc.fit_transform(y.reshape(-1, 1))
        return y_enc # output -> (n_samples, n_classes)
    
    def _init_weights(
        self, X: np.ndarray, y: np.ndarray,
        init_strategy: Literal['normal', 'corr', 'multistart'] = 'normal',
        n_starts: int = 5, search_steps: int = 50, lr: float = 1e-2,
    ):
        N, d = X.shape
        K = np.max(y) + 1

        if init_strategy == 'normal':
            if self.weights.size == 0:
                self.weights = self.rng_.standard_normal((d, K), dtype=np.float32)
            if self.bias.size == 0:
                self.bias = self.rng_.standard_normal((1, K), dtype=np.float32)
            return

        if init_strategy == 'corr':
            if self.weights.size != 0 and self.bias.size != 0:
                return
            # евклидова норма
            # denom = np.sum(X * X, axis=0)       # shape (d,)
            denom = np.float64(N)

            W = np.zeros((d, K), dtype=np.float64)
            b = np.zeros((1, K), dtype=np.float64)

            for k in range(K):
                t = (y == k).astype(np.float64) # 1 for class k, else 0
                # weights: elementwise division by per-feature squared norm
                # With centered X, X^T t == X^T (t - mean(t)), so no need to center t explicitly.
                numer = X.T @ t                           # shape (d,)
                W[:, k] = numer / denom

                # intercept: with centered features, LS gives b_k = mean(t^{(k)})
                b[0, k] = t.mean()

            if self.weights.size == 0:
                self.weights = W.astype(np.float32, copy=False)
            if self.bias.size == 0:
                self.bias = b.astype(np.float32, copy=False)
            return
        
        if init_strategy == 'multistart':
            best_loss = np.inf
            best_W, best_b = None, None

            for _ in range(n_starts):
                W = self.rng_.standard_normal((d, K), dtype=np.float32)
                b = self.rng_.standard_normal((1, K), dtype=np.float32)
                
                # short warmup
                w, b, loss = self._warmup(X, y, W, b, steps=search_steps, lr=lr)
                if loss < best_loss:
                    best_loss = loss
                    best_W, best_b = w, b

                self.weights = best_W
                self.bias    = best_b
            return
        
        raise ValueError("init_strategy must be 'normal' or 'corr'")
    
    def _warmup(self, X, y, W, b, steps=50, lr=1e-2):
        W = W.copy(); b = b.copy()
        for _ in range(steps):
            logits = np.matmul(X, W) + b
            loss   = self._loss_fn_opt(y, logits, reduction='mean')
            w_grad, b_grad = self._gradient(X, y, logits)
            W -= lr * w_grad
            b -= lr * b_grad
        return W, b, float(loss)
    
        
    def _softmax(self, X: np.array) -> np.array:
        Z = X - np.max(X, axis=1, keepdims=True)
        numerator = np.exp(Z)
        denominator = np.sum(numerator, axis=1, keepdims=True)
        softmax_probs = numerator / denominator
        return softmax_probs # -> (n_samples, n_classes)
    
    def forward(self, X):
        # (n_samples, n_features) * (n_features, n_classes)
        logits = np.matmul(X, self.weights) + self.bias # -> (n_samples, n_classes)
        return logits
    
    # def loss_fn_expanded(self, X, y_true):
    #     # (n_samples, n_features) * (n_features, n_classes) + (n_samples, 1) * (1, n_classes) = (n_samples, n_classes)
    #     logits = np.matmul(X, self.weights) + np.matmul(np.ones((X.shape[0], 1)), self.bias)
    #     exp_logits = np.exp(logits)
    #     logits_sum = np.sum(exp_logits, axis=1) # -> (n_samples, 1)
    #     # (n_samples, n_classes) * (n_samples, n_classes)
    #     true_class_logits = logits[np.arange(X.shape[0]), y_true]
    #     return np.mean(np.log(logits_sum) - true_class_logits)

    # def loss_fn(self, y_true, logits):
    #     log_probs = np.log(self.softmax(logits)) # -> (n_samples, classes)
    #     # y_true_ohe = self.create_onehot_target(y_true) # -> (n_samples, classes)
    #     # likelihood = (log_probs * y_true_ohe).sum(axis=1).mean()
    #     likelihood = (log_probs[np.arange(log_probs.shape[0]), y_true]).mean()
    #     return -likelihood
    
    def _loss_fn_opt(self, y_true, logits, reduction=None):
        lse = logsumexp(logits, axis=1, keepdims=True)
        nll = lse - logits
        loss = nll[np.arange(nll.shape[0]), y_true]
        if reduction == 'mean':
            loss = loss.mean()
        return loss
    
    def _rec_reset(self):
        self.rec_value = None
        self.rec_count = 0
        self.rec_history = []

    def _rec_update(self, xi, mode="off", ema_lambda=0.1):
        if mode == "off":
            return xi

        if self.rec_value is None:
            # инициализация последовательности
            self.rec_value = xi
            self.rec_count = 1
            return self.rec_value

        if mode == "mean":
            # running mean: Q_m = (1/m)*xi_m + (1 - 1/m)*Q_{m-1}
            self.rec_count += 1
            m = self.rec_count
            self.rec_value = (1.0/m)*xi + (1.0 - 1.0/m)*self.rec_value
            return self.rec_value

        if mode == "ema":
            # EMA: Q_m = λ xi_m + (1 - λ) Q_{m-1}
            self.rec_value = ema_lambda * xi + (1.0 - ema_lambda) * self.rec_value
            return self.rec_value

        return xi

    def _gradient(self, X, y_true, logits):
        y_prob = self._softmax(logits)
        y_prob[np.arange(y_prob.shape[0]), y_true] -= 1
        y_prob /= y_prob.shape[0]
        w_grad = np.matmul(X.T, y_prob)
        b_grad = y_prob.sum(axis=0, keepdims=True)
        return w_grad, b_grad
    

    def calc_margins(self, X, y_true, plot: bool = False, eps=1e-3, **kwargs):
        logits = self.forward(X)
        true_logits = logits[np.arange(X.shape[0]), y_true]
        logits[np.arange(logits.shape[0]), y_true] = -np.inf
        false_logits = logits.max(axis=1)
        margins = true_logits - false_logits

        if plot:
            
            sorted_idx = np.argsort(margins)
            sorted_margins = margins[sorted_idx]
            
            line_kwargs      = {'lw': 2}
            pos_fill_kwargs  = {'alpha': 0.25, 'color': 'tab:green'}
            neg_fill_kwargs  = {'alpha': 0.25, 'color': 'tab:red'}
            zero_fill_kwargs = {'alpha': 0.25, 'color': 'gold'}

            # masks
            if eps > 0.0:
                mask_zero = np.abs(sorted_margins) <= eps
                mask_pos  = sorted_margins >  eps
                mask_neg  = sorted_margins < -eps
            else:
                mask_zero = np.zeros_like(sorted_margins, dtype=bool)
                mask_pos  = sorted_margins > 0
                mask_neg  = sorted_margins < 0

            plt.figure(figsize=(12, 7))
            # line
            plot_idx = np.arange(sorted_margins.shape[0])
            plt.plot(plot_idx, sorted_margins, **line_kwargs)
            plt.axhline(0.0, color='black', lw=1, alpha=0.7)

            if np.any(mask_neg):
                plt.fill_between(plot_idx, sorted_margins, 0.0, where=mask_neg, interpolate=True, **neg_fill_kwargs)
            if np.any(mask_zero):
                plt.fill_between(plot_idx, sorted_margins, 0.0, where=mask_zero, interpolate=True, **zero_fill_kwargs)
            if np.any(mask_pos):
                plt.fill_between(plot_idx, sorted_margins, 0.0, where=mask_pos, interpolate=True, **pos_fill_kwargs)

            plt.xlabel("sample index (sorted)")
            plt.ylabel("margin")
            plt.title("Margin curve with signed areas")
            plt.tight_layout()
            plt.show()

        return margins
    
    # TODO: сделать сэмплер с разной логикой выбора сложных случаев
    # 1) -abs(margins) - для любых (правильных или нет) случаев с малой долей уверенности
    # 2) -margins вместо -abs(margins) - для точно неправильно классифицированных случаев
    def _margin_sampling_probs(
        self, X, y, use_abs: bool = True, tau: float = 0.2, min_prob: float = 0.01
    ):
        margins = self.calc_margins(X, y)

        diff = -np.abs(margins) if use_abs else -margins
        scores = diff / max(tau, 1e-8)
        probs = self._softmax(scores.reshape(1, -1)).squeeze()

        floor = min_prob / X.shape[0]
        probs = (1.0 - min_prob) * probs + floor

        return probs

In [ ]:
logreg = LogRegNumpy(early_stop=True)
# logreg._init_weights(X_train_scaled, y_train_enc, 'multistart')

In [ ]:
logreg.fit(
    X_train_scaled, y_train_enc,
    learning_rate=0.01, total_steps=X_train_scaled.shape[0]*3, init_strategy='corr',
    batch_size=X_train_scaled.shape[0], momentum=0.99, l2=0.0, optim_step=True, verbose=True, rec_mode='ema', ema_lambda=0.01,
    sampling_mode='uniform', shuffle=True, sampling_tau=0.2, sampling_min_prob=0.01, refresh_rate=200,
    steps_per_epoch=100
)

In [ ]:
plt.plot(logreg.loss_values)

In [ ]:
plt.plot(logreg.rec_history)

In [ ]:
margins = logreg.calc_margins(X_train_scaled, y_train_enc, plot=True, eps=1.0)

In [ ]:
preds_train = logreg.predict(X_train_scaled)
preds_test  = logreg.predict(X_test_scaled)

In [ ]:
print(f'Train accuracy: {(preds_train == y_train_enc).mean()*100.:.4f}')
print(f'Test accuracy: {(preds_test == y_test_enc).mean()*100.:.4f}')